In [ ]:
import qokit
import numpy as np
from itertools import combinations

# Simulation of QAOA applied to a general problem

In this tutorial, we show to get QAOA objective for a problem that does not have a directly implemented high-level API like LABS or MaxCut. In particular, we will consider the Sherrington-Kirkpatrick (SK) model given by the following objective:
$$
G(z) = -\sum_{1\leq i <j\leq N}J^{(N)}_{ij}z_iz_j
$$
where $J^{(N)}_{ij} \sim \mathcal{N}(\mu(N), \sigma^2)$.

To simulate the QAOA on SK model, we need to generate a problem representation. The format used by QOKit is a of tuples, where each tuple defines a summand and contains indices of the Pauli Zs in the product along with the coefficient. For example, if terms = [(0.5, (0,1)), (-1, (0,1,2,3)), (1,(1,2))], the Hamiltonian is
$$
0.5z_0z_1 - z_0z_1z_2z_3 + z_1z_2
$$

In [ ]:
N = 4
np.random.seed(10)
terms = [(np.random.normal(), spin_pair) for spin_pair in combinations(range(N), r=2)]

## Create simulator

`simclass` is a class that you should use to create the simulator.
There are multiple simulators in `qokit.fur` module.
You can choose to use `gpu` or `python` simulator and the following notebook should run without error

In [ ]:
simclass = qokit.fur.choose_simulator(name='auto')
sim = simclass(N, terms=terms)

In [ ]:
### Get precomputed diagonal cost vector and cache if desired

In [ ]:
sim.get_cost_diagonal()

## Simulate QAOA for parameters

Simulator returns a `result` object which may be different depending on the type of simulator you use.
If you want to use it directly, you have to know exactly which simulator you are using

In [ ]:
p = 3
gamma, beta = np.random.rand(2, 3)
_result = sim.simulate_qaoa(gamma, beta) # Result depends on the type of simulator. 

## Get simulation results

### Get statevector

In [ ]:
sv = sim.get_statevector(_result)
sv

### Get probabilities

The simulator will calculate probabilities for you. This may be done in-place, overwriting the data of statevector to avoid allocation of a separate array

In [ ]:
probs = sim.get_probabilities(_result)
probs.sum()

This will overwrite the `_result` if using GPU simulation. Subsequent calls to `get_statevector` return invalid data.

In [ ]:
probs = sim.get_probabilities(_result, preserve_state=False)
sv2 = sim.get_statevector(_result)
print("Using numpy") if np.allclose(sv, sv2) else print("Yohoo, I'm using a memory-economic simulator!")

### Get expectation value

For numpy version, the simulator effectively does `sv.dot(costs)`, where costs is the precomputed diagonal.

You may specify your own observable vector which will be used instead of the diagonal hamiltonian.

In [ ]:
e = sim.get_expectation(_result)
costs_inv = 1/sim.get_cost_diagonal()
e_inv = sim.get_expectation(_result, costs=costs_inv)
print("Expectation of C:", e)
print("1/(Expectation of 1/C): ", 1/e_inv)

### Get overlap

Returns the overlap with ground states, which are states corresponding to minimum of C. 
You may specify the corresponding costs as well. Additionally, prodive `indices` if you want to get probability for particular state.

In [ ]:
overlap = sim.get_overlap(_result)
print("Ground state overlap:", overlap)
# Below we test that for positive-valued cost function, the maximum can be achieved 
# by either inverting the values, or negating the values.
costs_abs = np.abs(sim.get_cost_diagonal())
print("Overlap with ground state for absolute cost:", sim.get_overlap(_result, costs=costs_abs))
overlap_inv = sim.get_overlap(_result, costs=1/costs_abs)
print("Overlap with highest state (inverted costs):", overlap_inv)
overlap_neg = sim.get_overlap(_result, costs=-costs_abs)
print("Overlap with highest state (negative):", overlap_neg)
assert overlap_inv == overlap_neg, "You may have values of mixed sign in your cost."

Overlap may be calculated in-place as well, which may make subsequent calls like `sim.get_expectation()` return incorrect values.
The default behavior is to copy, to provide consistent between simulator types.

If you only need one value for each `result`, you can pass `preserve_state=False` to reduce the memory usage.

In [ ]:
# specify state indices
overlap_03 = sim.get_overlap(_result, indices=[0, 3])
probs = sim.get_probabilities(_result)
assert overlap_03 == probs[[0, 3]].sum(), "This is a bug, please report it"

---

In [ ]:
#The code provided by Nadav in the morning on 1st July.

import qokit
import numpy as np
from itertools import combinations
import scipy
import networkx as nx
import time
from matplotlib import pyplot as plt

def compute_graph_colors(G, solution_index):
    N = G.number_of_nodes()
    vert_colors = list(map(lambda c : "green" if c == '1' else "blue", f"{int(solution_index):0{N}b}"))
    vert_colors.reverse()
    edge_colors = [("red" if vert_colors[u] != vert_colors[v] else "black") for u,v in G.edges()]
    return (vert_colors, edge_colors)

def print_result(N, terms, gamma, beta, adjective = ""):
    p = len(gamma)
    print(f"{adjective}Gamma: {gamma}")
    print(f"{adjective}Beta: {beta}")
    print(f"{adjective}Objective: {-inv_max_cut_objective(N, p, terms)(np.hstack([gamma, beta]))}")
    print()

def print_probabilities(G, terms, gamma, beta, threshold = 0, only_likely = False, include_cost = True):
    N = G.number_of_nodes()
    probs = get_probabilities(N, terms, gamma, beta)
    for element in np.column_stack((np.array(range(2**N)), probs)):
        [index, prob] = element
        likely_str = ""
        if (prob > threshold):
            likely_str = " <- Likely outcome"

        cost_str = ""
        if (include_cost):
            contained_verts = list(map(lambda c : c == '1', f"{int(index):0{N}b}"))
            contained_verts.reverse()
            cut_count = evaluate_cut(G, contained_verts)
            cost_str = f" ({cut_count})"

        if (not only_likely):
            print(f"{int(index):0{N}b}{cost_str} - {prob}{likely_str}")
        else:
            if (prob > threshold):
                print(f"{int(index):0{N}b}{cost_str} - {prob}")

def print_state_vector(G, terms, gamma, beta, include_cost = True):
    N = G.number_of_nodes()
    sv = get_state_vector(N, terms, gamma, beta)
    for element in np.column_stack((np.array(range(2**N)), sv)):
        [index, amplitude] = element
        cost_str = ""
        if (include_cost):
            contained_verts = list(map(lambda c : c == '1', f"{int(index):0{N}b}"))
            contained_verts.reverse()
            cut_count = evaluate_cut(G, contained_verts)
            cost_str = f" ({cut_count})"
        
        print(f"{int(index):0{N}b}{cost_str} - {amplitude}")

def plot_expectation(expectations, N, p, start_time):
    def make_time_relative(input):
        time, expectation = input
        return (time - start_time, expectation)

    time_relative_expectations = list(map(make_time_relative, expectations))
    plt.scatter(*zip(*time_relative_expectations))
    plt.title(f"MaxCut (N = {N}, p = {p})")
    plt.xlabel("Time (seconds)")
    plt.ylabel("QAOA Expectation")
    plt.show()

# TODO: Create plot with both expectation and measurements
def plot_measurements(measurements, N, p, start_time):
    def make_time_relative(input):
        time, measurement = input
        return (time - start_time, measurement)

    time_relative_measurements = list(map(make_time_relative, measurements))
    plt.scatter(*zip(*time_relative_measurements))
    plt.title(f"MaxCut (N = {N}, p = {p})")
    plt.xlabel("Time (seconds)")
    plt.ylabel("Measurement (best of 10)")
    plt.show()

def random_graph(N, prob_connect = 0.7):
    A = np.random.choice([0, 1], (N, N), p=[1 - prob_connect, prob_connect])
    np.fill_diagonal(A, 0)  # No self-loops
    A = np.triu(A)  # Use only the upper triangle
    A += A.T  # Make the matrix symmetric
    return (A, nx.from_numpy_array(A))

def max_cut_terms_for_graph(G):
    return list(map((lambda edge : (-0.5, edge)), G.edges)) + [((G.number_of_edges()/2.0), ())]

def optimal_solution_with_index(sim):
    costs = sim.get_cost_diagonal()
    solution = max(costs)
    solution_index = np.where(costs == solution)[0][0]
    return (solution, solution_index)

def evaluate_cut(G, bool_verts):
    cuts = 0
    for (u, v) in G.edges():
        if (bool_verts[u] != bool_verts[v]):
            cuts += 1
    return cuts

def get_simulator(N, terms, sim_or_none = None):
    if (sim_or_none is None):
        simclass = qokit.fur.choose_simulator(name='auto')
        return simclass(N, terms=terms)
    else:
        return sim_or_none
    
def get_result(N, terms, gamma, beta, sim = None, result = None):
    if (result is None):
        simulator = get_simulator(N, terms, sim)
        return simulator.simulate_qaoa(gamma, beta)
    else:
        return result
    
def get_simulator_and_result(N, terms, gamma, beta, sim = None, result = None):
    simulator = get_simulator(N, terms, sim)
    if (result is None):
        result = get_result(N, terms, gamma, beta, simulator)
    return (simulator, result)

def get_probabilities(N, terms, gamma, beta, sim = None, result = None):
    simulator, result = get_simulator_and_result(N, terms, gamma, beta, sim, result)
    return simulator.get_probabilities(result, preserve_state = True)

def get_expectation(N, terms, gamma, beta, sim = None, result = None):
    simulator, result = get_simulator_and_result(N, terms, gamma, beta, sim, result)
    return simulator.get_expectation(result, preserve_state = True)

def get_state_vector(N, terms, gamma, beta):
    simclass = qokit.fur.choose_simulator(name='auto')
    sim = simclass(N, terms=terms)
    _result = sim.simulate_qaoa(gamma, beta)
    return sim.get_statevector(_result)

def inv_max_cut_objective(N, p, terms, expectations = None, measurements = None, sim = None):
    def f(*args):
        gamma, beta = args[0][:p], args[0][p:]
        current_time = time.time()
        simulator = get_simulator(N, terms, sim)
        probs = get_probabilities(N, terms, gamma, beta, sim)
        costs = simulator.get_cost_diagonal()
        expectation = np.dot(costs, probs)

        if (expectations != None):
            expectations.append((current_time, expectation))

        if (measurements != None):
            measurement = max(np.random.choice(costs, 10, p=probs))
            measurements.append((current_time, measurement))
        
        return -expectation
    return f

def optimize(N, terms, init_gamma, init_beta, expectations = None, measurements = None, sim = None):
    if  (expectations != None):
        expectations.clear()
    if (measurements != None):
        measurements.clear()
    
    p = len(init_gamma)
    assert len(init_beta) == p, "Gamma and Beta must have the same length"
    init_freq = np.hstack([init_gamma, init_beta])

    res = scipy.optimize.minimize(inv_max_cut_objective(N, p, terms, expectations, measurements, sim), init_freq, method = 'COBYLA', options={'rhobeg': 0.01/N})

    gamma, beta = res.x[:p], res.x[p:]
    return (gamma, beta)

# TODO: Compare to randomly trying cuts (and keeping best)
def run_experiment(seed, N, p):
    np.random.seed(seed)
    (A, G) = random_graph(N, 0.5) 
    terms = max_cut_terms_for_graph(G)
    sim = get_simulator(N, terms)
    expectations = []
    measurements = []

    solution, solution_index = optimal_solution_with_index(sim)
    print(f"Optimal Solution: {solution_index:0{N}b} - {solution}")
    vert_colors, edge_colors = compute_graph_colors(G, solution_index)

    init_gamma, init_beta = np.random.rand(2, p)
    print_result(N, terms, init_gamma, init_beta, "Initial ")

    start_time = time.time()
    gamma, beta = optimize(N, terms, init_gamma, init_beta, expectations, measurements, sim)
    end_time = time.time()
    print(f"Time to optimize: {end_time - start_time} seconds\n")

    print_result(N, terms, gamma, beta, "Final ")
    print_probabilities(G, terms, gamma, beta, threshold=0.5/N, only_likely=True)
    print()
    plot_expectation(expectations, N, p, start_time)
    plot_measurements(measurements, N, p, start_time)

    nx.draw(G, nx.circular_layout(G), node_color=vert_colors, edge_color=edge_colors, with_labels=True)

def collect_state_vectors(num_vecs, G, p):
    N = G.number_of_nodes()
    terms = max_cut_terms_for_graph(G)
    qaoa_state_vectors = []

    for _ in range(num_vecs):
        init_gamma, init_beta = np.random.rand(2, p)
        gamma, beta = optimize(N, terms, init_gamma, init_beta)
        qaoa_state_vectors.append(get_state_vector(N, terms, gamma, beta))

    return qaoa_state_vectors
        
#(_, G) = random_graph(N=7)
#collect_state_vectors(10, G, p=3)
seed = int(input('seed = '))
N = int(input('N = '))
p = int(input('p = '))

run_experiment(seed, N, p)

In [ ]:
from qiskit import QuantumCircuit, Aer, transpile, assemble, execute
#←from qiskit import QuantumProgram
from qiskit.visualization import plot_histogram
#←from qiskit.tools.visualization import plot_histogram
import math
def Rx(qc, theta, q):
    qc.rx(theta, q)#←qc.u3(theta, -math.pi/2, math.pi/2, q)
def Rz(qc, phi, q):
    qc.rz(phi, q)#←qc.u1(phi, q)
def Rzz(qc, phi, q1, q2):
    qc.cx(q1, q2)
    Rz(qc, -2.0*phi, q2)
    qc.cx(q1, q2)
'''
def Rx(qci, theta, q):
    qci.u3(theta, -math.pi/2, math.pi/2, q)
def Rz(qci, phi, q):
    qci.u1(phi, q)
def Rzz(qci, phi, q1, q2):
    qci.cx(q1, q2)
    Rz(qci, -2.0*phi, q2)
    qci.cx(q1, q2)
'''
#qp = QuantumProgram()
cn = "Qp12-5"
ns = 2
tm = 10
dt = 0.1
#qr = qp.create_quantum_register("qr", ns)
#cr = qp.create_classical_register("cr", ns)
step = math.floor(tm / dt)
hz = [-1.0, -1.0]
J = -1.0
hx = -1.0
qc = QuantumCircuit(ns, ns)
#←qc = qp.create_circuit(cn, [qr], [cr])
for n in range(ns):
    qc.h(n)
'''
for n in range(0, ns):
    qc.h(qr[n])
'''
for n in range(step): #←for n in range(0, step):
    s = n / step
    for i in range(ns): #←for i in range(0, ns):
        Rx(qc, -2.0*(1-s)*hx*dt, i) #←Rx(qc, -2.0*(1-s)*hx*dt, qr[i])
        Rz(qc, -2.0*s*hz[i]*dt, i) #←Rz(qc, -2.0*s*hz[i]*dt, qr[i])
        for j in range(i+1, ns): #←for j in range(0, ns):
            #←if i < j:
            Rzz(qc, s*J*dt, i, j) #←Rzz(qc, s*J*dt, qr[i], qr[j])
#measurement
for n in range(ns): #←for n in range(0, ns):
    qc.measure(n, ns-1-n) #←qc.measure(qr[ns-1-n], cr[n]) 

#using the simulator
simulator = Aer.get_backend('qasm_simulator')
compiled_circuit = transpile(qc, simulator)
qobj = assemble(compiled_circuit, shots = 1000)
result = execute(qc, simulator, shots=1000).result()

#display the outcome
counts = result.get_counts()
print(counts)
plot_histogram(counts)
'''
r = qp.execute(cn, timeout = 600, shots = 1000)
print(r.get_data(cn))
plot_histogram(r.get_counts(cn))
'''

In [ ]:
#イジングハミルトニアンの断熱量子計算ゲートの基本プロトタイプ: Basic prototype of an adiabatic quantum computation gate for Ising Hamiltonian.

from qiskit import QuantumCircuit, Aer, transpile, assemble, execute
from qiskit.visualization import plot_histogram
import math

def Rx(qc, theta, q):
    qc.rx(theta, q)

def Rz(qc, phi, q):
    qc.rz(phi, q)

def Rzz(qc, phi, q1, q2):
    qc.cx(q1, q2)
    Rz(qc, -2.0 * phi, q2)
    qc.cx(q1, q2)

# 定数設定
cn = "Qp12-5"
ns = 2
tm = 10
dt = 0.1
step = math.floor(tm / dt)
hz = [-1.0, -1.0]
J = -1.0
hx = -1.0

# 量子回路の作成
qc = QuantumCircuit(ns, ns)
for n in range(ns):
    qc.h(n)

for n in range(step):
    s = n / step
    for i in range(ns):
        Rx(qc, -2.0 * (1-s) * hx * dt, i)
        Rz(qc, -2.0 * s * hz[i] * dt, i)
        for j in range(i+1, ns):
            Rzz(qc, s * J * dt, i, j)

# 測定
for n in range(ns):
    qc.measure(n, ns-1-n)

# シミュレータの使用
simulator = Aer.get_backend('qasm_simulator')
compiled_circuit = transpile(qc, simulator)
qobj = assemble(compiled_circuit, shots=1000)
result = execute(qc, simulator, shots=1000).result()

# 結果の表示
counts = result.get_counts()
print(counts)
plot_histogram(counts)


In [ ]:
#結合定数Jが負でのスピン整列: Spin alignment with negative coupling constant J

from qiskit import QuantumCircuit, Aer, transpile, assemble, execute #←from qiskit import QuantumProgram
from qiskit.visualization import plot_histogram #←from qiskit.tools.visualization import plot_histogram
import math
def Rx(qci, theta, q):
    qc.rx(theta, q) #←qci.u3(theta, -math.pi/2, math.pi/2, q)
def Rz(qc, phi, q):
    qc.rz(phi, q)
def Rzz(qci, phi, q1, q2):
    qc.cx(q1, q2) #←qci.cx(q1, q2)
    Rz(qci, -2.0*phi, q2)
    qc.cx(q1, q2) #←qci.cx(q1, q2)
#qp = QuantumProgram()
cn = "Qp12-6"
ns = 2
tm = 10
dt = 0.1
step = math.floor(tm / dt)
J = -1.0
hx = -1.0
#qr = qp.create_quantum_register("qr", ns)
#cr = qp.create_classical_register("cr", ns)

#making quantum circuits
qc = QuantumCircuit(ns, ns) #←qc = qp.create_circuit(cn, [qr], [cr])
for n in range(ns): #←for n in range(0, ns):
    qc.h(n) #←qc.h(qr[n])
#step = math.floor(tm/dt)
#J = -1.0
#hx = -1.0
for n in range(step): #←for n in range(0, step):
    s = n / step
    for i in range(ns): #←for i in range(0, ns):
        Rx(qc, -2.0*(1-s)*hx*dt, i) #←Rx(qc, -2.0*(1-s)*hx*dt, qr[i])
        for j in range(i+1, ns): #←for j in range(0, ns):
            #if i < j:
                Rzz(qc, s*J*dt, i, j) #←Rzz(qc, s*J*dt, qr[i], qr[j])

#measurement
for n in range(ns): #←for n in range(0, ns):
    qc.measure(n, ns-1-n)#←qc.measure(qr[ns-1-n], cr[n])

#using the simulator
simulator = Aer.get_backend('qasm_simulator')
compiled_circuit = transpile(qc, simulator)
qobj = assemble(compiled_circuit, shots = 1000)
result = execute(qc, simulator, shots = 1000).result()
#r = qp.execute(cn, timeout = 600, shots = 1000)

#display the outcome
counts = result.get_counts()
print(counts)
plot_histogram(counts)
#print(r.get_data(cn))
#plot_histogram(r.get_counts(cn))

In [ ]:
#The code provided by Nadav in the evening on 1st July.

import typing
import qokit
import numpy as np
import scipy
import networkx as nx
import time
from matplotlib import pyplot as plt
from qokit.fur.qaoa_simulator_base import QAOAFastSimulatorBase, TermsType
import qokit.maxcut as mc


def compute_graph_colors(G: nx.Graph, solution_index: int) -> tuple[list[str], list[str]]:
    N = G.number_of_nodes()
    vert_colors = list(map(lambda c: "green" if c == "1" else "blue", f"{int(solution_index):0{N}b}"))
    vert_colors.reverse()
    edge_colors = [("red" if vert_colors[u] != vert_colors[v] else "black") for u, v in G.edges()]
    return (vert_colors, edge_colors)


def print_result(N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, adjective: str = "") -> None:
    p = len(gamma)
    print(f"{adjective}Gamma: {gamma}")
    print(f"{adjective}Beta: {beta}")
    print(f"{adjective}Objective: {-inv_max_cut_objective(N, p, terms)(np.hstack([gamma, beta]))}")
    print()


def evaluate_cut(G: nx.Graph, bool_verts: list[bool]) -> int:
    cuts = 0
    for u, v in G.edges():
        if bool_verts[u] != bool_verts[v]:
            cuts += 1
    return cuts


def print_probabilities(
    G: nx.Graph, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, threshold: float = 0, only_likely: bool = False, include_cost: bool = True
) -> None:
    N = G.number_of_nodes()
    probs = get_probabilities(N, terms, gamma, beta)
    for element in np.column_stack((np.array(range(2**N)), probs)):
        [index, prob] = element
        likely_str = ""
        if prob > threshold:
            likely_str = " <- Likely outcome"

        cost_str = ""
        if include_cost:
            contained_verts = list(map(lambda c: c == "1", f"{int(index):0{N}b}"))
            contained_verts.reverse()
            cut_count = evaluate_cut(G, contained_verts)
            cost_str = f" ({cut_count})"

        if not only_likely:
            print(f"{int(index):0{N}b}{cost_str} - {prob}{likely_str}")
        else:
            if prob > threshold:
                print(f"{int(index):0{N}b}{cost_str} - {prob}")


def print_state_vector(G: nx.Graph, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, include_cost: bool = True) -> None:
    N = G.number_of_nodes()
    sv = get_state_vector(N, terms, gamma, beta)
    for element in np.column_stack((np.array(range(2**N)), sv)):
        [index, amplitude] = element
        cost_str = ""
        if include_cost:
            contained_verts = list(map(lambda c: c == "1", f"{int(index):0{N}b}"))
            contained_verts.reverse()
            cut_count = evaluate_cut(G, contained_verts)
            cost_str = f" ({cut_count})"

        print(f"{int(index):0{N}b}{cost_str} - {amplitude}")


def plot_expectation(expectations: list[float], N: int, p: int, start_time: float) -> None:
    def make_time_relative(input: tuple[float, float]) -> tuple[float, float]:
        time, expectation = input
        return (time - start_time, expectation)

    time_relative_expectations = list(map(make_time_relative, expectations))
    plt.scatter(*zip(*time_relative_expectations))
    plt.title(f"MaxCut (N = {N}, p = {p})")
    plt.xlabel("Time (seconds)")
    plt.ylabel("QAOA Expectation")
    plt.show()


def plot_measurements(measurements: list[float], N: int, p: int, start_time: float) -> None:
    def make_time_relative(input: tuple[float, float]) -> tuple[float, float]:
        time, measurement = input
        return (time - start_time, measurement)

    time_relative_measurements = list(map(make_time_relative, measurements))
    plt.scatter(*zip(*time_relative_measurements))
    plt.title(f"MaxCut (N = {N}, p = {p})")
    plt.xlabel("Time (seconds)")
    plt.ylabel("Measurement (best of 10)")
    plt.show()


def optimal_solution_with_index(sim: QAOAFastSimulatorBase) -> tuple[float, int]:
    costs = sim.get_cost_diagonal()
    solution = max(costs)
    solution_index = np.where(costs == solution)[0][0]
    return (solution, solution_index)


def get_simulator(N: int, terms: TermsType, sim_or_none: QAOAFastSimulatorBase | None = None) -> QAOAFastSimulatorBase:
    if sim_or_none is None:
        simclass = qokit.fur.choose_simulator(name="auto")
        return simclass(N, terms=terms)
    else:
        return sim_or_none


def get_result(
    N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, sim: QAOAFastSimulatorBase | None = None, result: np.ndarray | None = None
) -> np.ndarray:
    if result is None:
        simulator = get_simulator(N, terms, sim)
        return simulator.simulate_qaoa(gamma, beta)
    else:
        return result


def get_simulator_and_result(
    N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, sim: QAOAFastSimulatorBase | None = None, result: np.ndarray | None = None
) -> tuple[QAOAFastSimulatorBase, np.ndarray]:
    simulator = get_simulator(N, terms, sim)
    if result is None:
        result = get_result(N, terms, gamma, beta, simulator)
    return (simulator, result)


def get_probabilities(
    N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, sim: QAOAFastSimulatorBase | None = None, result: np.ndarray | None = None
) -> np.ndarray:
    simulator, result = get_simulator_and_result(N, terms, gamma, beta, sim, result)
    return simulator.get_probabilities(result, preserve_state=True)


def get_expectation(
    N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, sim: QAOAFastSimulatorBase | None = None, result: np.ndarray | None = None
) -> float:
    simulator, result = get_simulator_and_result(N, terms, gamma, beta, sim, result)
    return simulator.get_expectation(result, preserve_state=True)


def get_state_vector(N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray) -> np.ndarray:
    simclass = qokit.fur.choose_simulator(name="auto")
    sim = simclass(N, terms=terms)
    _result = sim.simulate_qaoa(gamma, beta)
    return sim.get_statevector(_result)


# Evaluates the objective function, and if expectations or measurements is passed in, appends to them
def inv_max_cut_objective(
    N: int, p: int, terms: TermsType, expectations: list[float] | None = None, measurements: list[float] | None = None, sim: QAOAFastSimulatorBase | None = None
) -> typing.Callable:
    def f(*args) -> float:
        gamma, beta = args[0][:p], args[0][p:]
        current_time = time.time()
        simulator = get_simulator(N, terms, sim)
        probs = get_probabilities(N, terms, gamma, beta, sim)
        costs = simulator.get_cost_diagonal()
        expectation = np.dot(costs, probs)

        if expectations != None:
            expectations.append((current_time, expectation))

        if measurements != None:
            measurement = max(np.random.choice(costs, 10, p=probs))
            measurements.append((current_time, measurement))

        return -expectation

    return f


# Given Eising terms and initial parameters, runs scipy.optimize.minimize and returns optimized parameters
# If expectations or measurements is passed in, populates them (this harms runtime)
def optimize(
    N: int,
    terms: TermsType,
    init_gamma: np.ndarray,
    init_beta: np.ndarray,
    expectations: list[float] | None = None,
    measurements: list[float] | None = None,
    sim: QAOAFastSimulatorBase | None = None,
) -> tuple[np.ndarray, np.ndarray]:
    if expectations != None:
        expectations.clear()
    if measurements != None:
        measurements.clear()

    p = len(init_gamma)
    assert len(init_beta) == p, "Gamma and Beta must have the same length"
    init_freq = np.hstack([init_gamma, init_beta])

    res = scipy.optimize.minimize(inv_max_cut_objective(N, p, terms, expectations, measurements, sim), init_freq, method="COBYLA", options={"rhobeg": 0.01 / N})

    gamma, beta = res.x[:p], res.x[p:]
    return (gamma, beta)


# Generates a random graph with N vertices and runs MaxCut QAOA on this graph, plotting results
def run_experiment(seed: int, N: int, p: int) -> None:
    np.random.seed(seed)
    G = nx.erdos_renyi_graph(N, 0.5, seed=seed)  # Random graph w/ 0.5 edge probability
    terms = mc.get_maxcut_terms(G)
    sim = get_simulator(N, terms)
    expectations = []
    measurements = []

    # Compute and print solution by brute force
    solution, solution_index = optimal_solution_with_index(sim)
    print(f"Optimal Solution: {solution_index:0{N}b} - {solution}")
    vert_colors, edge_colors = compute_graph_colors(G, solution_index)

    # Random starting parameters
    init_gamma, init_beta = np.random.rand(2, p)
    print_result(N, terms, init_gamma, init_beta, "Initial ")

    start_time = time.time()
    # If you don't want expectation and measurement plots, do not pass in parameters!
    gamma, beta = optimize(N, terms, init_gamma, init_beta, expectations, measurements, sim)
    end_time = time.time()
    print(f"Time to optimize: {end_time - start_time} seconds\n")

    print_result(N, terms, gamma, beta, "Final ")
    print_probabilities(G, terms, gamma, beta, threshold=0.5 / N, only_likely=True)
    print()
    plot_expectation(expectations, N, p, start_time)
    plot_measurements(measurements, N, p, start_time)

    nx.draw(G, nx.circular_layout(G), node_color=vert_colors, edge_color=edge_colors, with_labels=True)


seed = int(input('seed = '))
N = int(input('N = '))
p = int(input('p = '))

run_experiment(seed, N, p)

In [ ]:
#結合定数Jが正でのスピン整列: Spin alignment with positive coupling constant J

from qiskit import QuantumCircuit, Aer, transpile, assemble, execute #←from qiskit import QuantumProgram
from qiskit.visualization import plot_histogram #←from qiskit.tools.visualization import plot_histogram
import math
def Rx(qci, theta, q):
    qc.rx(theta, q) #←qci.u3(theta, -math.pi/2, math.pi/2, q)
def Rz(qc, phi, q):
    qc.rz(phi, q)
def Rzz(qci, phi, q1, q2):
    qc.cx(q1, q2) #←qci.cx(q1, q2)
    Rz(qci, -2.0*phi, q2)
    qc.cx(q1, q2) #←qci.cx(q1, q2)
#qp = QuantumProgram()
cn = "Qp12-6"
ns = 2
tm = 10
dt = 0.1
step = math.floor(tm / dt)
J = 1.0
hx = -1.0
#qr = qp.create_quantum_register("qr", ns)
#cr = qp.create_classical_register("cr", ns)

#making quantum circuits
qc = QuantumCircuit(ns, ns) #←qc = qp.create_circuit(cn, [qr], [cr])
for n in range(ns): #←for n in range(0, ns):
    qc.h(n) #←qc.h(qr[n])
#step = math.floor(tm/dt)
#J = -1.0
#hx = -1.0
for n in range(step): #←for n in range(0, step):
    s = n / step
    for i in range(ns): #←for i in range(0, ns):
        Rx(qc, -2.0*(1-s)*hx*dt, i) #←Rx(qc, -2.0*(1-s)*hx*dt, qr[i])
        for j in range(i+1, ns): #←for j in range(0, ns):
            #if i < j:
                Rzz(qc, s*J*dt, i, j) #←Rzz(qc, s*J*dt, qr[i], qr[j])

#measurement
for n in range(ns): #←for n in range(0, ns):
    qc.measure(n, ns-1-n)#←qc.measure(qr[ns-1-n], cr[n])

#using the simulator
simulator = Aer.get_backend('qasm_simulator')
compiled_circuit = transpile(qc, simulator)
qobj = assemble(compiled_circuit, shots = 1000)
result = execute(qc, simulator, shots = 1000).result()
#r = qp.execute(cn, timeout = 600, shots = 1000)

#display the outcome
counts = result.get_counts()
print(counts)
plot_histogram(counts)
#print(r.get_data(cn))
#plot_histogram(r.get_counts(cn))

In [ ]:
#負の結合定数jと負の縦磁場hzのあるイジングハミルトニアンの断熱量子計算ゲート: Adiabatic quantum computing gate of Ising Hamiltonian with a negative coupling constant J and a negative longitudinal magnetic field

from qiskit import QuantumCircuit, Aer, transpile, assemble, execute #←from qiskit import QuantumProgram
from qiskit.visualization import plot_histogram #←from qiskit.tools.visualization import plot_histogram
import math
def Rx(qc, theta, q): #←def Rx(qci, theta, q):
    qc.u(theta, -math.pi/2, math.pi/2, q) #←qci.u3(theta, -math.pi/2, math.pi/2, q)
def Rz(qc, phi, q): #←def Rz(qci, phi, q):
    qc.rz(phi, q) #←qci.u1(phi, q)
def Rzz(qc, phi, q1, q2): #←def Rzz(qci, phi, q1, q2):
    qc.cx(q1, q2) #←qci.cx(q1, q2)
    Rz(qc, -2.0 * phi, q2) #←Rz(qci, -2.0*phi, q2)
    qc.cx(q1, q2) #←qci.cx(q1, q2)
#qp = QuantumProgram()

#Create a quantum circuit
cn = "Qp12-8"
ns = 2
tm = 10
dt = 0.1
qr = QuantumCircuit(ns, ns)#←qr = qp.create_quantum_register("qr", ns)
#cr = qp.create_classical_register("cr", ns)
#qc = qp.create_circuit(cn, [qr], [cr])

#Apply Hadamard gates to all qubits
for n in range(ns): #←for n in range(0, ns):
    qr.h(n) #←qc.h(qr[n])

step = math.floor(tm/dt)
hz = [-1, -1]
J = -1.0
hx = -1.0
for n in range(step): #←for n in range(0, step):
    s = n / step
    for i in range(ns): #←for i in range(0, ns):
        Rx(qr, -2.0*(1-s)*hx*dt, i)#←Rx(qc, -2.0*(1-s)*hx*dt, qr[i])
        Rz(qr, -2.0*s*hz[i]*dt, i)#←Rz(qc, -2.0*s*hz[i]*dt, qr[i])
        for j in range(ns): #←for j in range(0, ns):
            if i < j:
                Rzz(qc, s*J*dt, i, j) #←Rzz(qc, s*J*dt, qr[i], qr[j])

#Measure all qubits
for n in range(0, ns):
    qr.measure(ns-1-n, n) #←qc.measure(qr[ns-1-n], cr[n])

# Execute the circuit
backend = Aer.get_backend('qasm_simulator')
tqc = transpile(qr, backend)
qobj = assemble(tqc, shots=1000)
result = execute(qr, backend, shots=1000).result()

# Print and plot the result
counts = result.get_counts()
print(counts)
plot_histogram(counts)
#r = qp.execute(cn, timeout = 600, shots = 1000)
#print(r.get_data(cn))
#plot_histogram(r.get_counts(cn))

In [ ]:
#QAOA_N_vs_Runtime arranged by Tina

import typing
import qokit
import numpy as np
import scipy
import networkx as nx
import time
from matplotlib import pyplot as plt
from qokit.fur.qaoa_simulator_base import QAOAFastSimulatorBase, TermsType
import qokit.maxcut as mc

#
def compute_graph_colors(G: nx.Graph, solution_index: int) -> tuple[list[str], list[str]]:
    N = G.number_of_nodes()
    vert_colors = list(map(lambda c: "green" if c == "1" else "blue", f"{int(solution_index):0{N}b}"))
    vert_colors.reverse()
    edge_colors = [("red" if vert_colors[u] != vert_colors[v] else "black") for u, v in G.edges()]
    return (vert_colors, edge_colors)


def print_result(N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, adjective: str = "") -> None:
    p = len(gamma)
    # print(f"{adjective}Gamma: {gamma}")
    # print(f"{adjective}Beta: {beta}")
    # print(f"{adjective}Objective: {-inv_max_cut_objective(N, p, terms)(np.hstack([gamma, beta]))}")
    # print()


def evaluate_cut(G: nx.Graph, bool_verts: list[bool]) -> int:
    cuts = 0
    for u, v in G.edges():
        if bool_verts[u] != bool_verts[v]:
            cuts += 1
    return cuts


def print_probabilities(
    G: nx.Graph, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, threshold: float = 0, only_likely: bool = False, include_cost: bool = True
) -> None:
    N = G.number_of_nodes()
    probs = get_probabilities(N, terms, gamma, beta)
    for element in np.column_stack((np.array(range(2**N)), probs)):
        [index, prob] = element
        likely_str = ""
        if prob > threshold:
            likely_str = " <- Likely outcome"

        cost_str = ""
        if include_cost:
            contained_verts = list(map(lambda c: c == "1", f"{int(index):0{N}b}"))
            contained_verts.reverse()
            cut_count = evaluate_cut(G, contained_verts)
            cost_str = f" ({cut_count})"

        # if not only_likely:
        #     print(f"{int(index):0{N}b}{cost_str} - {prob}{likely_str}")
        # else:
        #     if prob > threshold:
        #         print(f"{int(index):0{N}b}{cost_str} - {prob}")


def print_state_vector(G: nx.Graph, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, include_cost: bool = True) -> None:
    N = G.number_of_nodes()
    sv = get_state_vector(N, terms, gamma, beta)
    for element in np.column_stack((np.array(range(2**N)), sv)):
        [index, amplitude] = element
        cost_str = ""
        if include_cost:
            contained_verts = list(map(lambda c: c == "1", f"{int(index):0{N}b}"))
            contained_verts.reverse()
            cut_count = evaluate_cut(G, contained_verts)
            cost_str = f" ({cut_count})"

        # print(f"{int(index):0{N}b}{cost_str} - {amplitude}")


def plot_expectation(expectations: list[float], N: int, p: int, start_time: float) -> None:
    def make_time_relative(input: tuple[float, float]) -> tuple[float, float]:
        time, expectation = input
        return (time - start_time, expectation)

    time_relative_expectations = list(map(make_time_relative, expectations))
    # plt.scatter(*zip(*time_relative_expectations))
    # plt.title(f"MaxCut (N = {N}, p = {p})")
    # plt.xlabel("Time (seconds)")
    # plt.ylabel("QAOA Expectation")
    # plt.show()


def plot_measurements(measurements: list[float], N: int, p: int, start_time: float) -> None:
    def make_time_relative(input: tuple[float, float]) -> tuple[float, float]:
        time, measurement = input
        return (time - start_time, measurement)

    time_relative_measurements = list(map(make_time_relative, measurements))
    # plt.scatter(*zip(*time_relative_measurements))
    # plt.title(f"MaxCut (N = {N}, p = {p})")
    # plt.xlabel("Time (seconds)")
    # plt.ylabel("Measurement (best of 10)")
    # plt.show()


def optimal_solution_with_index(sim: QAOAFastSimulatorBase) -> tuple[float, int]:
    costs = sim.get_cost_diagonal()
    solution = max(costs)
    solution_index = np.where(costs == solution)[0][0]
    return (solution, solution_index)


def get_simulator(N: int, terms: TermsType, sim_or_none: QAOAFastSimulatorBase | None = None) -> QAOAFastSimulatorBase:
    if sim_or_none is None:
        simclass = qokit.fur.choose_simulator(name="auto")
        return simclass(N, terms=terms)
    else:
        return sim_or_none


def get_result(
    N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, sim: QAOAFastSimulatorBase | None = None, result: np.ndarray | None = None
) -> np.ndarray:
    if result is None:
        simulator = get_simulator(N, terms, sim)
        return simulator.simulate_qaoa(gamma, beta)
    else:
        return result


def get_simulator_and_result(
    N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, sim: QAOAFastSimulatorBase | None = None, result: np.ndarray | None = None
) -> tuple[QAOAFastSimulatorBase, np.ndarray]:
    simulator = get_simulator(N, terms, sim)
    if result is None:
        result = get_result(N, terms, gamma, beta, simulator)
    return (simulator, result)


def get_probabilities(
    N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, sim: QAOAFastSimulatorBase | None = None, result: np.ndarray | None = None
) -> np.ndarray:
    simulator, result = get_simulator_and_result(N, terms, gamma, beta, sim, result)
    return simulator.get_probabilities(result, preserve_state=True)


def get_expectation(
    N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, sim: QAOAFastSimulatorBase | None = None, result: np.ndarray | None = None
) -> float:
    simulator, result = get_simulator_and_result(N, terms, gamma, beta, sim, result)
    return simulator.get_expectation(result, preserve_state=True)


def get_state_vector(N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray) -> np.ndarray:
    simclass = qokit.fur.choose_simulator(name="auto")
    sim = simclass(N, terms=terms)
    _result = sim.simulate_qaoa(gamma, beta)
    return sim.get_statevector(_result)


# Evaluates the objective function, and if expectations or measurements is passed in, appends to them
def inv_max_cut_objective(
    N: int, p: int, terms: TermsType, expectations: list[float] | None = None, measurements: list[float] | None = None, sim: QAOAFastSimulatorBase | None = None
) -> typing.Callable:
    def f(*args) -> float:
        gamma, beta = args[0][:p], args[0][p:]
        current_time = time.time()
        simulator = get_simulator(N, terms, sim)
        probs = get_probabilities(N, terms, gamma, beta, sim)
        costs = simulator.get_cost_diagonal()
        expectation = np.dot(costs, probs)

        if expectations != None:
            expectations.append((current_time, expectation))

        if measurements != None:
            measurement = max(np.random.choice(costs, 10, p=probs))
            measurements.append((current_time, measurement))

        return -expectation

    return f


# Given Eising terms and initial parameters, runs scipy.optimize.minimize and returns optimized parameters
# If expectations or measurements is passed in, populates them (this harms runtime)
def optimize(
    N: int,
    terms: TermsType,
    init_gamma: np.ndarray,
    init_beta: np.ndarray,
    expectations: list[float] | None = None,
    measurements: list[float] | None = None,
    sim: QAOAFastSimulatorBase | None = None,
) -> tuple[np.ndarray, np.ndarray]:
    if expectations != None:
        expectations.clear()
    if measurements != None:
        measurements.clear()

    p = len(init_gamma)
    assert len(init_beta) == p, "Gamma and Beta must have the same length"
    init_freq = np.hstack([init_gamma, init_beta])

    res = scipy.optimize.minimize(inv_max_cut_objective(N, p, terms, expectations, measurements, sim), init_freq, method="COBYLA", options={"rhobeg": 0.01 / N})

    gamma, beta = res.x[:p], res.x[p:]
    return (gamma, beta)


# Generates a random graph with N vertices and runs MaxCut QAOA on this graph, plotting results
def run_experiment(seed: int, N: int, p: int) -> None:
    np.random.seed(seed)
    G = nx.erdos_renyi_graph(N, 0.5, seed=seed)  # Random graph w/ 0.5 edge probability
    terms = mc.get_maxcut_terms(G)
    sim = get_simulator(N, terms)
    expectations = []
    measurements = []

    # Compute and print solution by brute force
    solution, solution_index = optimal_solution_with_index(sim)
    # print(f"Optimal Solution: {solution_index:0{N}b} - {solution}")
    vert_colors, edge_colors = compute_graph_colors(G, solution_index)

    # Random starting parameters
    init_gamma, init_beta = np.random.rand(2, p)
    # print_result(N, terms, init_gamma, init_beta, "Initial ")

    start_time = time.time()
    # If you don't want expectation and measurement plots, do not pass in parameters!
    gamma, beta = optimize(N, terms, init_gamma, init_beta, expectations, measurements, sim)
    end_time = time.time()
    # print(f"Time to optimize: {end_time - start_time} seconds\n")

    # print_result(N, terms, gamma, beta, "Final ")
    # print_probabilities(G, terms, gamma, beta, threshold=0.5 / N, only_likely=True)
    # print()
    # plot_expectation(expectations, N, p, start_time)
    # plot_measurements(measurements, N, p, start_time)

    # nx.draw(G, nx.circular_layout(G), node_color=vert_colors, edge_color=edge_colors, with_labels=True)
    return (end_time - start_time)

# List of Ns
Ns = list(range(1, 11))
runtimes = []

# Run experiments for each N and collect runtimes
for i in Ns:
    runtime = run_experiment(seed = 0, N = i, p = 5)
    runtimes.append(runtime)

# Plotting runtime vs N
plt.plot(Ns, runtimes, marker='o')
plt.title('Runtime vs N')
plt.xlabel('Number of Nodes (N)')
plt.ylabel('Runtime (seconds)')
plt.grid(True)
plt.show()

In [ ]:
#QAOA_simulator coded by Nadav

import typing
import qokit
import numpy as np
import scipy
import time
from qokit.fur.qaoa_simulator_base import QAOAFastSimulatorBase, TermsType

'''
This will serve as a module for QAOA simulation functionalities. 

The main function is QAOA_run, which uses QAOA with specified parameters for the ising model 
that it is passed. 

Most other functions are written only for the purpose of QAOA_run to use them. 
'''

def get_simulator(N: int, terms: TermsType, sim_or_none: QAOAFastSimulatorBase | None = None) -> QAOAFastSimulatorBase:
    if sim_or_none is None:
        simclass = qokit.fur.choose_simulator(name="auto")
        return simclass(N, terms=terms)
    else:
        return sim_or_none


def get_result(
    N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, sim: QAOAFastSimulatorBase | None = None, result: np.ndarray | None = None
) -> np.ndarray:
    if result is None:
        simulator = get_simulator(N, terms, sim)
        return simulator.simulate_qaoa(gamma, beta)
    else:
        return result


def get_simulator_and_result(
    N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, sim: QAOAFastSimulatorBase | None = None, result: np.ndarray | None = None
) -> tuple[QAOAFastSimulatorBase, np.ndarray]:
    simulator = get_simulator(N, terms, sim)
    if result is None:
        result = get_result(N, terms, gamma, beta, simulator)
    return (simulator, result)


def get_state(N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, sim: QAOAFastSimulatorBase | None = None, result: np.ndarray | None = None):
    simulator, result = get_simulator_and_result(N, terms, gamma, beta, sim, result)
    return simulator.get_statevector(result)


def get_probabilities(
    N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, sim: QAOAFastSimulatorBase | None = None, result: np.ndarray | None = None
) -> np.ndarray:
    simulator, result = get_simulator_and_result(N, terms, gamma, beta, sim, result)
    return simulator.get_probabilities(result, preserve_state=True)


def get_expectation(
    N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, sim: QAOAFastSimulatorBase | None = None, result: np.ndarray | None = None
) -> float:
    simulator, result = get_simulator_and_result(N, terms, gamma, beta, sim, result)
    return simulator.get_expectation(result, preserve_state=True)


def get_overlap(
    N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, sim: QAOAFastSimulatorBase | None = None, result: np.ndarray | None = None
) -> float:
    simulator, result = get_simulator_and_result(N, terms, gamma, beta, sim, result)
    return simulator.get_overlap(result, preserve_state=True)


def inverse_objective_function(ising_model: TermsType, N: int, p: int, mixer: str, states: list[np.ndarray] | None) -> typing.Callable:
    def inverse_objective(*args) -> float:
        gamma, beta = args[0][:p], args[0][p:]
        simulator, result = get_simulator_and_result(N, ising_model, gamma, beta)
        expectation = get_expectation(N, ising_model, gamma, beta, simulator, result)

        if states is not None:
            states.append(get_state(N, ising_model, gamma, beta, simulator, result))

        return -expectation

    return inverse_objective


def QAOA_run(
    ising_model: TermsType,
    N: int,
    p: int,
    init_gamma: np.ndarray,
    init_beta: np.ndarray,
    optimizer_method: str = 'COBYLA',
    optimizer_options: dict | None = None,
    mixer: str = "x",  # Using a different mixer is not yet supported
    states: list[np.ndarray] | None = None,
) -> dict:
    init_freq = np.hstack([init_gamma, init_beta]) 
    #call_counter = [0]  #not needed; use result.nit instead 

    start_time = time.time()
    result = scipy.optimize.minimize(
        inverse_objective_function(ising_model, N, p, mixer, states), init_freq, method=optimizer_method, options=optimizer_options
    ) 
    #the above returns a scipy optimization result object that has multiple attributes
    #result.x gives the optimal solutionsol.success #bool whether algorithm succeeded
    #result.message #message of why algorithms terminated
    #result.nfev is number of iterations used (here, number of QAOA calls)
    end_time = time.time()

    gamma, beta = result.x[:p], result.x[p:]

    return {
        "gamma": gamma,
        "beta": beta,
        "state": get_state(N, ising_model, gamma, beta),
        "expectation": get_expectation(N, ising_model, gamma, beta),
        "overlap": get_overlap(N, ising_model, gamma, beta),
        "runtime": end_time - start_time,  # measured in seconds
        "num_QAOA_calls": result.nfev,
        "classical_opt_success": result.success,
        "scipy_opt_message": result.message
    }

In [ ]:
#QAOA_plots with my arrangement written by Nadav

import typing
import qokit
import numpy as np
import scipy
import networkx as nx
import time
from matplotlib import pyplot as plt
from qokit.fur.qaoa_simulator_base import QAOAFastSimulatorBase, TermsType
import qokit.maxcut as mc


def compute_graph_colors(G: nx.Graph, solution_index: int) -> tuple[list[str], list[str]]:
    N = G.number_of_nodes()
    vert_colors = list(map(lambda c: "green" if c == "1" else "blue", f"{int(solution_index):0{N}b}"))
    vert_colors.reverse()
    edge_colors = [("red" if vert_colors[u] != vert_colors[v] else "black") for u, v in G.edges()]
    return (vert_colors, edge_colors)


def print_result(N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, adjective: str = "") -> None:
    p = len(gamma)
    print(f"{adjective}Gamma: {gamma}")
    print(f"{adjective}Beta: {beta}")
    print(f"{adjective}Objective: {-inv_max_cut_objective(N, p, terms)(np.hstack([gamma, beta]))}")
    print()


def evaluate_cut(G: nx.Graph, bool_verts: list[bool]) -> int:
    cuts = 0
    for u, v in G.edges():
        if bool_verts[u] != bool_verts[v]:
            cuts += 1
    return cuts
#comment_as_the_first_practice

def print_probabilities(
    G: nx.Graph, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, threshold: float = 0, only_likely: bool = False, include_cost: bool = True
) -> None:
    N = G.number_of_nodes()
    probs = get_probabilities(N, terms, gamma, beta)
    for element in np.column_stack((np.array(range(2**N)), probs)):
        [index, prob] = element
        likely_str = ""
        if prob > threshold:
            likely_str = " <- Likely outcome"

        cost_str = ""
        if include_cost:
            contained_verts = list(map(lambda c: c == "1", f"{int(index):0{N}b}"))
            contained_verts.reverse()
            cut_count = evaluate_cut(G, contained_verts)
            cost_str = f" ({cut_count})"

        if not only_likely:
            print(f"{int(index):0{N}b}{cost_str} - {prob}{likely_str}")
        else:
            if prob > threshold:
                print(f"{int(index):0{N}b}{cost_str} - {prob}")


def print_state_vector(G: nx.Graph, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, include_cost: bool = True) -> None:
    N = G.number_of_nodes()
    sv = get_state_vector(N, terms, gamma, beta)
    for element in np.column_stack((np.array(range(2**N)), sv)):
        [index, amplitude] = element
        cost_str = ""
        if include_cost:
            contained_verts = list(map(lambda c: c == "1", f"{int(index):0{N}b}"))
            contained_verts.reverse()
            cut_count = evaluate_cut(G, contained_verts)
            cost_str = f" ({cut_count})"

        print(f"{int(index):0{N}b}{cost_str} - {amplitude}")


def plot_expectation(expectations: list[float], N: int, p: int, start_time: float) -> None:
    def make_time_relative(input: tuple[float, float]) -> tuple[float, float]:
        time, expectation = input
        return (time - start_time, expectation)

    time_relative_expectations = list(map(make_time_relative, expectations))
    plt.scatter(*zip(*time_relative_expectations))
    plt.title(f"MaxCut (N = {N}, p = {p})")
    plt.xlabel("Time (seconds)")
    plt.ylabel("QAOA Expectation")
    plt.show()


def plot_measurements(measurements: list[float], N: int, p: int, start_time: float) -> None:
    def make_time_relative(input: tuple[float, float]) -> tuple[float, float]:
        time, measurement = input
        return (time - start_time, measurement)

    time_relative_measurements = list(map(make_time_relative, measurements))
    plt.scatter(*zip(*time_relative_measurements))
    plt.title(f"MaxCut (N = {N}, p = {p})")
    plt.xlabel("Time (seconds)")
    plt.ylabel("Measurement (best of 10)")
    plt.show()


def optimal_solution_with_index(sim: QAOAFastSimulatorBase) -> tuple[float, int]:
    costs = sim.get_cost_diagonal()
    solution = max(costs)
    solution_index = np.where(costs == solution)[0][0]
    return (solution, solution_index)


def get_simulator(N: int, terms: TermsType, sim_or_none: QAOAFastSimulatorBase | None = None) -> QAOAFastSimulatorBase:
    if sim_or_none is None:
        simclass = qokit.fur.choose_simulator(name="auto")
        return simclass(N, terms=terms)
    else:
        return sim_or_none


def get_result(
    N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, sim: QAOAFastSimulatorBase | None = None, result: np.ndarray | None = None
) -> np.ndarray:
    if result is None:
        simulator = get_simulator(N, terms, sim)
        return simulator.simulate_qaoa(gamma, beta)
    else:
        return result


def get_simulator_and_result(
    N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, sim: QAOAFastSimulatorBase | None = None, result: np.ndarray | None = None
) -> tuple[QAOAFastSimulatorBase, np.ndarray]:
    simulator = get_simulator(N, terms, sim)
    if result is None:
        result = get_result(N, terms, gamma, beta, simulator)
    return (simulator, result)


def get_probabilities(
    N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, sim: QAOAFastSimulatorBase | None = None, result: np.ndarray | None = None
) -> np.ndarray:
    simulator, result = get_simulator_and_result(N, terms, gamma, beta, sim, result)
    return simulator.get_probabilities(result, preserve_state=True)


def get_expectation(
    N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, sim: QAOAFastSimulatorBase | None = None, result: np.ndarray | None = None
) -> float:
    simulator, result = get_simulator_and_result(N, terms, gamma, beta, sim, result)
    return simulator.get_expectation(result, preserve_state=True)


def get_state_vector(N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray) -> np.ndarray:
    simclass = qokit.fur.choose_simulator(name="auto")
    sim = simclass(N, terms=terms)
    _result = sim.simulate_qaoa(gamma, beta)
    return sim.get_statevector(_result)


# Evaluates the objective function, and if expectations or measurements is passed in, appends to them
def inv_max_cut_objective(
    N: int, p: int, terms: TermsType, expectations: list[float] | None = None, measurements: list[float] | None = None, sim: QAOAFastSimulatorBase | None = None
) -> typing.Callable:
    def f(*args) -> float:
        gamma, beta = args[0][:p], args[0][p:]
        current_time = time.time()
        simulator = get_simulator(N, terms, sim)
        probs = get_probabilities(N, terms, gamma, beta, sim)
        costs = simulator.get_cost_diagonal()
        expectation = np.dot(costs, probs)

        if expectations != None:
            expectations.append((current_time, expectation))

        if measurements != None:
            measurement = max(np.random.choice(costs, 10, p=probs))
            measurements.append((current_time, measurement))

        return -expectation

    return f


# Given Eising terms and initial parameters, runs scipy.optimize.minimize and returns optimized parameters
# If expectations or measurements is passed in, populates them (this harms runtime)
def optimize(
    N: int,
    terms: TermsType,
    init_gamma: np.ndarray,
    init_beta: np.ndarray,
    expectations: list[float] | None = None,
    measurements: list[float] | None = None,
    sim: QAOAFastSimulatorBase | None = None,
) -> tuple[np.ndarray, np.ndarray]:
    if expectations != None:
        expectations.clear()
    if measurements != None:
        measurements.clear()

    p = len(init_gamma)
    assert len(init_beta) == p, "Gamma and Beta must have the same length"
    init_freq = np.hstack([init_gamma, init_beta])

    res = scipy.optimize.minimize(inv_max_cut_objective(N, p, terms, expectations, measurements, sim), init_freq, method="COBYLA", options={"rhobeg": 0.01 / N})

    gamma, beta = res.x[:p], res.x[p:]
    return (gamma, beta)


# Generates a random graph with N vertices and runs MaxCut QAOA on this graph, plotting results
def run_experiment(seed: int, N: int, p: int) -> None:
    np.random.seed(seed)
    G = nx.erdos_renyi_graph(N, 0.5, seed=seed)  # Random graph w/ 0.5 edge probability
    terms = mc.get_maxcut_terms(G)
    sim = get_simulator(N, terms)
    expectations = []
    measurements = []

    # Compute and print solution by brute force
    solution, solution_index = optimal_solution_with_index(sim)
    print(f"Optimal Solution: {solution_index:0{N}b} - {solution}")
    vert_colors, edge_colors = compute_graph_colors(G, solution_index)

    # Random starting parameters
    init_gamma, init_beta = np.random.rand(2, p)
    print_result(N, terms, init_gamma, init_beta, "Initial ")

    start_time = time.time()
    # If you don't want expectation and measurement plots, do not pass in parameters!
    gamma, beta = optimize(N, terms, init_gamma, init_beta, expectations, measurements, sim)
    end_time = time.time()
    print(f"Time to optimize: {end_time - start_time} seconds\n")

    print_result(N, terms, gamma, beta, "Final ")
    print_probabilities(G, terms, gamma, beta, threshold=0.5 / N, only_likely=True)
    print()
    plot_expectation(expectations, N, p, start_time)
    plot_measurements(measurements, N, p, start_time)

    nx.draw(G, nx.circular_layout(G), node_color=vert_colors, edge_color=edge_colors, with_labels=True)

seed = int(input('seed = '))
N = int(input('N = '))
p = int(input('p = '))

run_experiment(seed, N, p)

In [ ]:
#The experiment arranged by Kanon with ChatGPT
import qokit  # QAOAのシミュレーションに使用するqokitライブラリをインポート
import numpy as np  # 数値計算用のライブラリnumpyをインポート
from itertools import combinations  # 組み合わせを生成するitertoolsモジュールからcombinationsをインポート
import scipy  # 科学技術計算用ライブラリscipyをインポート
import networkx as nx  # グラフ理論を操作するnetworkxライブラリをインポート
import time  # 時間計測用のライブラリtimeをインポート
from matplotlib import pyplot as plt  # プロット用のmatplotlibのpyplotモジュールをインポート

expectations_along_the_way = []  # QAOA実行中の期待値の推移を記録するリスト
measurements_along_the_way = []  # QAOA実行中の測定結果の推移を記録するリスト
costs = []  # カットのコストを保持するリスト

def random_graph(N, prob_connect=0.7):
    """
    N個のノードを持つランダムなグラフを生成する関数。
    prob_connectの確率でノード間にエッジを生成する。

    Parameters:
    - N: グラフのノード数
    - prob_connect: ノード間にエッジを生成する確率

    Returns:
    - A: 隣接行列
    - G: networkxのグラフオブジェクト
    """
    A = np.random.choice([0, 1], (N, N), p=[1 - prob_connect, prob_connect])
    np.fill_diagonal(A, 0)  # 自己ループを禁止
    A = np.triu(A)  # 上三角部分のみ使用
    A += A.T  # 行列を対称にする
    return (A, nx.from_numpy_array(A))

def max_cut_terms_for_graph(G):
    """
    最大カット問題の項を生成する関数。
    グラフGのエッジごとに項を生成し、最後に全結合項を追加する。

    Parameters:
    - G: networkxのグラフオブジェクト

    Returns:
    - terms: 最大カット問題の項のリスト
    """
    return list(map((lambda edge: (-0.5, edge)), G.edges)) + [((G.number_of_edges() / 2.0), ())]

def print_result(N, terms, gamma, beta, adjective=""):
    """
    QAOAの実行結果を出力する関数。
    gamma, beta, 目的関数の値を表示する。

    Parameters:
    - N: グラフのノード数
    - terms: 最大カット問題の項のリスト
    - gamma: QAOAのγパラメータ
    - beta: QAOAのβパラメータ
    - adjective: 接頭辞として付ける形容詞（オプション）

    Returns:
    - None
    """
    p = len(gamma)
    print(f"{adjective}Gamma: {gamma}")
    print(f"{adjective}Beta: {beta}")
    print(f"{adjective}Objective: {-inv_max_cut_objective(N, p, terms)(np.hstack([gamma, beta]))}")
    print()

def evaluate_cut(G, bool_verts):
    """
    カットの数を評価する関数。
    bool_vertsで指定されたカットにおけるカットの数を数える。

    Parameters:
    - G: networkxのグラフオブジェクト
    - bool_verts: カットを表すブール値のリスト

    Returns:
    - cuts: カットの数
    """
    cuts = 0
    for (u, v) in G.edges():
        if (bool_verts[u] != bool_verts[v]):
            cuts += 1
    return cuts

def get_probabilities(N, terms, gamma, beta):
    """
    QAOAの確率分布を取得する関数。

    Parameters:
    - N: グラフのノード数
    - terms: 最大カット問題の項のリスト
    - gamma: QAOAのγパラメータ
    - beta: QAOAのβパラメータ

    Returns:
    - probs: 各状態の確率分布
    """
    simclass = qokit.fur.choose_simulator(name='auto')
    sim = simclass(N, terms=terms)
    _result = sim.simulate_qaoa(gamma, beta)
    return sim.get_probabilities(_result)

def get_expectation(N, terms, gamma, beta):
    """
    QAOAの期待値を取得する関数。

    Parameters:
    - N: グラフのノード数
    - terms: 最大カット問題の項のリスト
    - gamma: QAOAのγパラメータ
    - beta: QAOAのβパラメータ

    Returns:
    - expectation: QAOAの期待値
    """
    simclass = qokit.fur.choose_simulator(name='auto')
    sim = simclass(N, terms=terms)
    _result = sim.simulate_qaoa(gamma, beta)
    return sim.get_expectation(_result)

def print_probabilities(G, terms, gamma, beta, threshold=0, only_likely=False, include_cost=True):
    """
    QAOAの実行結果の確率分布を表示する関数。

    Parameters:
    - G: networkxのグラフオブジェクト
    - terms: 最大カット問題の項のリスト
    - gamma: QAOAのγパラメータ
    - beta: QAOAのβパラメータ
    - threshold: 表示する確率の閾値
    - only_likely: 閾値を超える確率のみ表示するかどうか
    - include_cost: カットのコストを表示するかどうか

    Returns:
    - None
    """
    N = G.number_of_nodes()
    probs = get_probabilities(N, terms, gamma, beta)
    for element in np.column_stack((np.array(range(2**N)), probs)):
        [index, prob] = element
        likely_str = ""
        if (prob > threshold):
            likely_str = " <- Likely outcome"

        cost_str = ""
        if (include_cost):
            contained_verts = list(map(lambda c: c == '1', f"{int(index):0{N}b}"))
            contained_verts.reverse()
            cut_count = evaluate_cut(G, contained_verts)
            cost_str = f" ({cut_count})"

        if (not only_likely):
            print(f"{int(index):0{N}b}{cost_str} - {prob}{likely_str}")
        else:
            if (prob > threshold):
                print(f"{int(index):0{N}b}{cost_str} - {prob}")

def get_state_vector(N, terms, gamma, beta):
    """
    QAOAの状態ベクトルを取得する関数。

    Parameters:
    - N: グラフのノード数
    - terms: 最大カット問題の項のリスト
    - gamma: QAOAのγパラメータ
    - beta: QAOAのβパラメータ

    Returns:
    - sv: QAOAの状態ベクトル
    """
    simclass = qokit.fur.choose_simulator(name='auto')
    sim = simclass(N, terms=terms)
    _result = sim.simulate_qaoa(gamma, beta)
    return sim.get_statevector(_result)

def print_state_vector(G, terms, gamma, beta, include_cost=True):
    """
    QAOAの状態ベクトルを表示する関数。

    Parameters:
    - G: networkxのグラフオブジェクト
    - terms: 最大カット問題の項のリスト
    - gamma: QAOAのγパラメータ
    - beta: QAOAのβパラメータ
    - include_cost: カットのコストを表示するかどうか

    Returns:
    - None
    """
    N = G.number_of_nodes()
    sv = get_state_vector(N, terms, gamma, beta)
    for element in np.column_stack((np.array(range(2**N)), sv)):
        [index, amplitude] = element
        cost_str = ""
        if (include_cost):
            contained_verts = list(map(lambda c: c == '1', f"{int(index):0{N}b}"))
            contained_verts.reverse()
            cut_count = evaluate_cut(G, contained_verts)
            cost_str = f" ({cut_count})"

        print(f"{int(index):0{N}b}{cost_str} - {amplitude}")

def set_costs(G):
    """
    グラフGのカットのコストを設定する関数。

    Parameters:
    - G: networkxのグラフオブジェクト

    Returns:
    - None
    """
    costs.clear()
    N = G.number_of_nodes()
    for index in range(2**N):
        contained_verts = list(map(lambda c: c == '1', f"{int(index):0{N}b}"))
        contained_verts.reverse()
        costs.append(evaluate_cut(G, contained_verts))

def inv_max_cut_objective(N, p, terms):
    """
    最大カット問題の目的関数の逆を定義する関数。

    Parameters:
    - N: グラフのノード数
    - p: QAOAのステップ数
    - terms: 最大カット問題の項のリスト

    Returns:
    - f: 目的関数の逆の関数
    """
    def f(*args):
        gamma, beta = args[0][:p], args[0][p:]
        current_time = time.time()

        expectation = get_expectation(N, terms, gamma, beta)
        expectations_along_the_way.append((current_time, expectation))

        if (len(costs) > 0):
            probs = get_probabilities(N, terms, gamma, beta)
            measurement = max(np.random.choice(costs, 10, p=probs))
            measurements_along_the_way.append((current_time, measurement))

        return -expectation
    return f

def optimize(N, terms, init_gamma, init_beta):
    """
    QAOAのパラメータを最適化する関数。

    Parameters:
    - N: グラフのノード数
    - terms: 最大カット問題の項のリスト
    - init_gamma: 初期のγパラメータ
    - init_beta: 初期のβパラメータ

    Returns:
    - gamma: 最適化されたγパラメータ
    - beta: 最適化されたβパラメータ
    """
    expectations_along_the_way.clear()
    measurements_along_the_way.clear()
    p = len(init_gamma)
    init_freq = np.hstack([init_gamma, init_beta])
    res = scipy.optimize.minimize(inv_max_cut_objective(N, p, terms), init_freq, method='COBYLA', options={'rhobeg': 0.01/N})
    gamma, beta = res.x[:p], res.x[p:]
    return (gamma, beta)

def plot_expectation(N, p, start_time):
    """
    QAOAの期待値の推移をプロットする関数。

    Parameters:
    - N: グラフのノード数
    - p: QAOAのステップ数
    - start_time: 実験開始時刻

    Returns:
    - None
    """
    def make_time_relative(input):
        time, expectation = input
        return (time - start_time, expectation)

    time_relative_expectations = list(map(make_time_relative, expectations_along_the_way))
    plt.scatter(*zip(*time_relative_expectations))
    plt.title(f"MaxCut (N = {N}, p = {p})")
    plt.xlabel("Time (seconds)")
    plt.ylabel("QAOA Expectation")
    plt.show()

def plot_measurements(N, p, start_time):
    """
    QAOAの測定結果の推移をプロットする関数。

    Parameters:
    - N: グラフのノード数
    - p: QAOAのステップ数
    - start_time: 実験開始時刻

    Returns:
    - None
    """
    def make_time_relative(input):
        time, measurement = input
        return (time - start_time, measurement)

    time_relative_measurements = list(map(make_time_relative, measurements_along_the_way))
    plt.scatter(*zip(*time_relative_measurements))
    plt.title(f"MaxCut (N = {N}, p = {p})")
    plt.xlabel("Time (seconds)")
    plt.ylabel("Measurement (best of 10)")
    plt.show()

def run_experiment(seed, N, p):
    """
    QAOAの実験を実行する関数。

    Parameters:
    - seed: 乱数シード
    - N: グラフのノード数
    - p: QAOAのステップ数

    Returns:
    - None
    """
    np.random.seed(seed)
    (A, G) = random_graph(N, 0.5)
    set_costs(G)
    solution = max(costs)
    solution_index = costs.index(solution)
    print(f"Optimal Solution: {solution_index:0{N}b} - {max(costs)}")
    vert_colors = list(map(lambda c: "green" if c == '1' else "blue", f"{int(solution_index):0{N}b}"))
    vert_colors.reverse()
    edge_colors = [("red" if vert_colors[u] != vert_colors[v] else "black") for u, v in G.edges()]

    terms = max_cut_terms_for_graph(G)
    init_gamma, init_beta = np.random.rand(2, p)

    print_result(N, terms, init_gamma, init_beta, "Initial ")

    start_time = time.time()
    gamma, beta = optimize(N, terms, init_gamma, init_beta)
    end_time = time.time()
    print(f"Time to optimize: {end_time - start_time} seconds\n")

    print_result(N, terms, gamma, beta, "Final ")
    print_probabilities(G, terms, gamma, beta, threshold=0.5/N, only_likely=True)
    print()
    plot_expectation(N, p, start_time)
    plot_measurements(N, p, start_time)

    nx.draw(G, nx.circular_layout(G), node_color=vert_colors, edge_color=edge_colors, with_labels=True)
    return  # ここで return 文を追加する

# 実験の実行（N=10, p=5）
run_experiment(seed=0, N=10, p=5)

def set_costs(G):
    """
    グラフGのカットのコストを設定する関数。

    Parameters:
    - G: networkxのグラフオブジェクト

    Returns:
    - None
    """
    global costs  # global宣言でcostsを関数外の変数として扱う
    costs = []
    N = G.number_of_nodes()
    for index in range(2**N):
        contained_verts = list(map(lambda c: c == '1', f"{int(index):0{N}b}"))
        contained_verts.reverse()
        costs.append(evaluate_cut(G, contained_verts))

def run_experiments_for_different_Ns(seed, N_values, p):
    expectations = []

    for N in N_values:
        np.random.seed(seed)
        (A, G) = random_graph(N, 0.5)
        set_costs(G)
        terms = max_cut_terms_for_graph(G)
        init_gamma, init_beta = np.random.rand(2, p)

        start_time = time.time()
        gamma, beta = optimize(N, terms, init_gamma, init_beta)
        end_time = time.time()
        print(f"N = {N}: Time to optimize = {end_time - start_time} seconds")

        expectation = get_expectation(N, terms, gamma, beta)
        expectations.append(expectation)

    return N_values, expectations

# Nの値を設定
N_values = [1,2,3,4,5,6,7,8,9,10,11]

# 実験を実行して期待値を取得
seed = 0  # 乱数シード
p = 5  # QAOAのステップ数
N_values, expectations = run_experiments_for_different_Ns(seed, N_values, p)

# 結果をプロット
plt.figure()
plt.plot(N_values, expectations, marker='o')
plt.title('QAOA Expectation vs. N')
plt.xlabel('N (Number of nodes)')
plt.ylabel('QAOA Expectation')
plt.grid(True)
plt.show()

In [ ]:
#負の結合定数jと正の縦磁場hzのあるイジングハミルトニアンの断熱量子計算ゲート: Adiabatic quantum computing gate of Ising Hamiltonian with a negative coupling constant J and a positive longitudinal magnetic field

from qiskit import QuantumCircuit, Aer, transpile, assemble, execute #←from qiskit import QuantumProgram
from qiskit.visualization import plot_histogram #←from qiskit.tools.visualization import plot_histogram
import math
def Rx(qc, theta, q): #←def Rx(qci, theta, q):
    qc.u(theta, -math.pi/2, math.pi/2, q) #←qci.u3(theta, -math.pi/2, math.pi/2, q)
def Rz(qc, phi, q): #←def Rz(qci, phi, q):
    qc.rz(phi, q) #←qci.u1(phi, q)
def Rzz(qc, phi, q1, q2): #←def Rzz(qci, phi, q1, q2):
    qc.cx(q1, q2) #←qci.cx(q1, q2)
    Rz(qc, -2.0 * phi, q2) #←Rz(qci, -2.0*phi, q2)
    qc.cx(q1, q2) #←qci.cx(q1, q2)
#qp = QuantumProgram()

#Create a quantum circuit
cn = "Qp12-9"
ns = 2
tm = 10
dt = 0.1
qr = QuantumCircuit(ns, ns)#←qr = qp.create_quantum_register("qr", ns)
#cr = qp.create_classical_register("cr", ns)
#qc = qp.create_circuit(cn, [qr], [cr])

#Apply Hadamard gates to all qubits
for n in range(ns): #←for n in range(0, ns):
    qr.h(n) #←qc.h(qr[n])

step = math.floor(tm/dt)
hz = [1, 1]
J = -1.0
hx = -1.0
for n in range(step): #←for n in range(0, step):
    s = n / step
    for i in range(ns): #←for i in range(0, ns):
        Rx(qr, -2.0*(1-s)*hx*dt, i)#←Rx(qc, -2.0*(1-s)*hx*dt, qr[i])
        Rz(qr, -2.0*s*hz[i]*dt, i)#←Rz(qc, -2.0*s*hz[i]*dt, qr[i])
        for j in range(ns): #←for j in range(0, ns):
            if i < j:
                Rzz(qc, s*J*dt, i, j) #←Rzz(qc, s*J*dt, qr[i], qr[j])

#Measure all qubits
for n in range(0, ns):
    qr.measure(ns-1-n, n) #←qc.measure(qr[ns-1-n], cr[n])

# Execute the circuit
backend = Aer.get_backend('qasm_simulator')
tqc = transpile(qr, backend)
qobj = assemble(tqc, shots=1000)
result = execute(qr, backend, shots=1000).result()

# Print and plot the result
counts = result.get_counts()
print(counts)
plot_histogram(counts)
#r = qp.execute(cn, timeout = 600, shots = 1000)
#print(r.get_data(cn))
#plot_histogram(r.get_counts(cn))

In [ ]:
#正の結合定数jと正の縦磁場hzのあるイジングハミルトニアンの断熱量子計算ゲート: Adiabatic quantum computing gate of Ising Hamiltonian with a positive coupling constant J and a positive longitudinal magnetic field

from qiskit import QuantumCircuit, Aer, transpile, assemble, execute #←from qiskit import QuantumProgram
from qiskit.visualization import plot_histogram #←from qiskit.tools.visualization import plot_histogram
import math
def Rx(qc, theta, q): #←def Rx(qci, theta, q):
    qc.u(theta, -math.pi/2, math.pi/2, q) #←qci.u3(theta, -math.pi/2, math.pi/2, q)
def Rz(qc, phi, q): #←def Rz(qci, phi, q):
    qc.rz(phi, q) #←qci.u1(phi, q)
def Rzz(qc, phi, q1, q2): #←def Rzz(qci, phi, q1, q2):
    qc.cx(q1, q2) #←qci.cx(q1, q2)
    Rz(qc, -2.0 * phi, q2) #←Rz(qci, -2.0*phi, q2)
    qc.cx(q1, q2) #←qci.cx(q1, q2)
#qp = QuantumProgram()

#Create a quantum circuit
cn = "Qp12-10"
ns = 2
tm = 10
dt = 0.1
qr = QuantumCircuit(ns, ns)#←qr = qp.create_quantum_register("qr", ns)
#cr = qp.create_classical_register("cr", ns)
#qc = qp.create_circuit(cn, [qr], [cr])

#Apply Hadamard gates to all qubits
for n in range(ns): #←for n in range(0, ns):
    qr.h(n) #←qc.h(qr[n])

step = math.floor(tm/dt)
hz = [1, 1]
J = 1.0
hx = -1.0
for n in range(step): #←for n in range(0, step):
    s = n / step
    for i in range(ns): #←for i in range(0, ns):
        Rx(qr, -2.0*(1-s)*hx*dt, i)#←Rx(qc, -2.0*(1-s)*hx*dt, qr[i])
        Rz(qr, -2.0*s*hz[i]*dt, i)#←Rz(qc, -2.0*s*hz[i]*dt, qr[i])
        for j in range(ns): #←for j in range(0, ns):
            if i < j:
                Rzz(qc, s*J*dt, i, j) #←Rzz(qc, s*J*dt, qr[i], qr[j])

#Measure all qubits
for n in range(0, ns):
    qr.measure(ns-1-n, n) #←qc.measure(qr[ns-1-n], cr[n])

# Execute the circuit
backend = Aer.get_backend('qasm_simulator')
tqc = transpile(qr, backend)
qobj = assemble(tqc, shots=1000)
result = execute(qr, backend, shots=1000).result()

# Print and plot the result
counts = result.get_counts()
print(counts)
plot_histogram(counts)
#r = qp.execute(cn, timeout = 600, shots = 1000)
#print(r.get_data(cn))
#plot_histogram(r.get_counts(cn))

In [ ]:
#正の結合定数jと負の縦磁場hzのあるイジングハミルトニアンの断熱量子計算ゲート: Adiabatic quantum computing gate of Ising Hamiltonian with a positive coupling constant J and a negative longitudinal magnetic field

from qiskit import QuantumCircuit, Aer, transpile, assemble, execute #←from qiskit import QuantumProgram
from qiskit.visualization import plot_histogram #←from qiskit.tools.visualization import plot_histogram
import math
def Rx(qc, theta, q): #←def Rx(qci, theta, q):
    qc.u(theta, -math.pi/2, math.pi/2, q) #←qci.u3(theta, -math.pi/2, math.pi/2, q)
def Rz(qc, phi, q): #←def Rz(qci, phi, q):
    qc.rz(phi, q) #←qci.u1(phi, q)
def Rzz(qc, phi, q1, q2): #←def Rzz(qci, phi, q1, q2):
    qc.cx(q1, q2) #←qci.cx(q1, q2)
    Rz(qc, -2.0 * phi, q2) #←Rz(qci, -2.0*phi, q2)
    qc.cx(q1, q2) #←qci.cx(q1, q2)
#qp = QuantumProgram()

#Create a quantum circuit
cn = "Qp12-11"
ns = 2
tm = 10
dt = 0.1
qr = QuantumCircuit(ns, ns)#←qr = qp.create_quantum_register("qr", ns)
#cr = qp.create_classical_register("cr", ns)
#qc = qp.create_circuit(cn, [qr], [cr])

#Apply Hadamard gates to all qubits
for n in range(ns): #←for n in range(0, ns):
    qr.h(n) #←qc.h(qr[n])

step = math.floor(tm/dt)
hz = [1, 1]
J = 1.0
hx = -1.0
for n in range(step): #←for n in range(0, step):
    s = n / step
    for i in range(ns): #←for i in range(0, ns):
        Rx(qr, -2.0*(1-s)*hx*dt, i)#←Rx(qc, -2.0*(1-s)*hx*dt, qr[i])
        Rz(qr, -2.0*s*hz[i]*dt, i)#←Rz(qc, -2.0*s*hz[i]*dt, qr[i])
        for j in range(ns): #←for j in range(0, ns):
            if i < j:
                Rzz(qc, s*J*dt, i, j) #←Rzz(qc, s*J*dt, qr[i], qr[j])

#Measure all qubits
for n in range(0, ns):
    qr.measure(ns-1-n, n) #←qc.measure(qr[ns-1-n], cr[n])

# Execute the circuit
backend = Aer.get_backend('qasm_simulator')
tqc = transpile(qr, backend)
qobj = assemble(tqc, shots=1000)
result = execute(qr, backend, shots=1000).result()

# Print and plot the result
counts = result.get_counts()
print(counts)
plot_histogram(counts)
#r = qp.execute(cn, timeout = 600, shots = 1000)
#print(r.get_data(cn))
#plot_histogram(r.get_counts(cn))